In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/


In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown


In [ ]:
import zipfile
file=zipfile.ZipFile('dogs-vs-cats.zip','r')
file.extractall()
file.close()

In [ ]:
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D
from keras.models import Sequential
import keras
import tensorflow as tf


In [ ]:
!pip install Augmentor
import Augmentor

In [ ]:
p=Augmentor.Pipeline('train/cats')
p.zoom(probability=0.3,min_factor=0.8,max_factor=1.5)
p.flip_top_bottom(probability=0.4)
p.flip_left_right(probability=0.4)
p.sample(1000)

Initialised with 10000 image(s) found.
Output directory set to train/cats/output.

Processing <PIL.Image.Image image mode=RGB size=241x230 at 0x7F6F246675D0>: 100%|██████████| 1000/1000 [00:06<00:00, 165.28 Samples/s]


In [ ]:
p=Augmentor.Pipeline('train/dogs')
p.zoom(probability=0.3,min_factor=0.8,max_factor=1.5)
p.flip_top_bottom(probability=0.4)
p.flip_left_right(probability=0.4)
p.sample(1000)

Initialised with 10000 image(s) found.
Output directory set to train/dogs/output.

Processing <PIL.Image.Image image mode=RGB size=439x367 at 0x7F6F267D9190>: 100%|██████████| 1000/1000 [00:07<00:00, 129.58 Samples/s]


In [ ]:
import shutil
import os

# Source directory (where the files are currently located)
source_dir = 'train/dogs/output'

# Destination directory (where you want to move the files)
destination_dir = 'train/dogs'

# Iterate over all files and subdirectories in the source directory
for filename in os.listdir(source_dir):
    # Construct the full file paths for the source and destination
    source_path = os.path.join(source_dir, filename)
    destination_path = os.path.join(destination_dir, filename)

    # Move the file
    shutil.move(source_path, destination_path)

print("All files moved successfully!")

All files moved successfully!


In [ ]:
import shutil
import os

# Source directory (where the files are currently located)
source_dir = 'train/cats/output'

# Destination directory (where you want to move the files)
destination_dir = 'train/cats'

# Iterate over all files and subdirectories in the source directory
for filename in os.listdir(source_dir):
    # Construct the full file paths for the source and destination
    source_path = os.path.join(source_dir, filename)
    destination_path = os.path.join(destination_dir, filename)

    # Move the file
    shutil.move(source_path, destination_path)

print("All files moved successfully!")

All files moved successfully!


In [ ]:
X_train=keras.utils.image_dataset_from_directory(
    directory='train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(255,255)
)

Found 22000 files belonging to 2 classes.


In [ ]:
X_test=keras.utils.image_dataset_from_directory(
    directory='test',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(255,255)
)


Found 5000 files belonging to 2 classes.


In [ ]:
len(X_test)

157

In [ ]:
def process(image,label):
  image=tf.cast(image/255.0,tf.float32)
  return image,label

In [ ]:
X_train=X_train.map(process)
X_test=X_test.map(process)

In [ ]:
model=Sequential()

In [ ]:
model.add(Conv2D(32,(3,3),padding='same',activation='relu',input_shape=(255,255,3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128,(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(X_train,validation_data=X_test,epochs=5)

Epoch 1/5
688/688 ━━━━━━━━━━━━━━━━━━━━ 62s 77ms/step - accuracy: 0.5492 - loss: 0.7779 - val_accuracy: 0.6830 - val_loss: 0.5973
Epoch 2/5
688/688 ━━━━━━━━━━━━━━━━━━━━ 75s 77ms/step - accuracy: 0.7173 - loss: 0.5584 - val_accuracy: 0.7702 - val_loss: 0.4969
Epoch 3/5
688/688 ━━━━━━━━━━━━━━━━━━━━ 80s 74ms/step - accuracy: 0.8021 - loss: 0.4184 - val_accuracy: 0.7660 - val_loss: 0.5971
Epoch 4/5
688/688 ━━━━━━━━━━━━━━━━━━━━ 81s 74ms/step - accuracy: 0.8988 - loss: 0.2504 - val_accuracy: 0.7752 - val_loss: 0.8215
Epoch 5/5
688/688 ━━━━━━━━━━━━━━━━━━━━ 51s 74ms/step - accuracy: 0.9545 - loss: 0.1207 - val_accuracy: 0.7774 - val_loss: 0.9052


In [ ]:

def catordog(m1):
   if m1>0.5:
      return 'dog'
   else:
      return 'cat'

In [ ]:
import pickle

In [ ]:
pickle.dump(model,open('catvsdog.pkl','wb'))

In [ ]:
model2=pickle.load(open('catvsdog.pkl','rb'))

In [ ]:
import cv2
def image_to_array(path):
  img=cv2.imread(path)
  img=cv2.resize(img,(255,255))
  img=img.reshape((1,255,255,3))
  img=img/255.0
  return img

In [ ]:
resent_model=Sequential()

In [ ]:
pretrained_model=tf.keras.applications.ResNet50(include_top=False,
                                                input_shape=(255,255,3),
                                                pooling='avg',classes=2,
                                                weights='imagenet')
for layers in pretrained_model.layers:
  layers.trainable=False
resent_model.add(pretrained_model)
resent_model.add(Flatten())
resent_model.add(Dense(32,activation='relu'))
resent_model.add(Dense(1,activation='sigmoid'))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [ ]:
resent_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
resent_model.fit(X_train,validation_data=X_test,epochs=5)

Epoch 1/5
688/688 ━━━━━━━━━━━━━━━━━━━━ 108s 138ms/step - accuracy: 0.5719 - loss: 0.6795 - val_accuracy: 0.6330 - val_loss: 0.6403
Epoch 2/5
688/688 ━━━━━━━━━━━━━━━━━━━━ 89s 129ms/step - accuracy: 0.6267 - loss: 0.6438 - val_accuracy: 0.6434 - val_loss: 0.6295
Epoch 3/5
688/688 ━━━━━━━━━━━━━━━━━━━━ 136s 121ms/step - accuracy: 0.6273 - loss: 0.6411 - val_accuracy: 0.6002 - val_loss: 0.6668
Epoch 4/5
688/688 ━━━━━━━━━━━━━━━━━━━━ 147s 129ms/step - accuracy: 0.6298 - loss: 0.6397 - val_accuracy: 0.6122 - val_loss: 0.6429
Epoch 5/5
688/688 ━━━━━━━━━━━━━━━━━━━━ 89s 129ms/step - accuracy: 0.6406 - loss: 0.6295 - val_accuracy: 0.6560 - val_loss: 0.6198


In [ ]:
img=image_to_array('download.jpeg')

In [ ]:
resent_pred=resent_model.predict(img)
normal_pred=model.predict(img)
print("Resent model prediction : ",catordog(resent_pred))
print("Nomral model predicition : ",catordog(normal_pred))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Resent model prediction :  cat
Nomral model predicition :  cat


In [ ]:
import pickle
pickle.dump(resent_pred,open('catvsdog_resent.pkl','wb'))
pickle.dump(normal_pred,open('catvsdog_normal.pkl','wb'))